#### <a id='top'></a>

# Homework 2: Data Visualization and Probability Analysis 
***

**Name**: Justin Yara

***

This assignment is due on Canvas by **MIDNIGHT on Monday September 14**. Your solutions to theoretical questions should be done in Markdown directly below the associated question.  Your solutions to computational questions should include any specified Python code and results as well as written commentary on your conclusions.  Remember that you are encouraged to discuss the problems with your classmates, but **you must write all code and solutions on your own**.

**NOTES**: 

- Any relevant data sets should be available under the **Data** module on Canvas. To make life easier on the graders if they need to run your code, do not change the relative path names here. Instead, move the files around on your computer.
- If you're not familiar with typesetting math directly into Markdown then by all means, do your work on paper first and then typeset it later.  Remember that there is a [reference guide](https://math.meta.stackexchange.com/questions/5020/mathjax-basic-tutorial-and-quick-reference) linked on Canvas on writing math in Markdown. **All** of your written commentary, justifications and mathematical work should be in Markdown.
- Because you can technically evaluate notebook cells is a non-linear order, it's a good idea to do Kernel $\rightarrow$ Restart & Run All as a check before submitting your solutions.  That way if we need to run your code you will know that it will work as expected. 
- It is **bad form** to make your reader interpret numerical output from your code.  If a question asks you to compute some value from the data you should show your code output **AND** write a summary of the results in Markdown directly below your code. 
- 95 points of this assignment are in problems.  The remaining 5 are for neatness, style, and overall exposition of both code and text.
- This probably goes without saying, but... For any question that asks you to calculate something, you **must show all work and justify your answers to receive credit**. Sparse or nonexistent work will receive sparse or nonexistent credit. 

---
**Shortcuts:**  [Problem 1](#p1) | [Problem 2](#p2) | [Problem 3](#p3) |

---

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

[Back to top](#top)

<br>

<a id='p1'></a>

## (15 points) Problem 1: Theory (Median Distance)
***

One way we conceptualize many data science questions is asking for the "best choice" of some parameter on data set.  We should be able to justify that our measures of centrality should in some way be the "best" ways to represent the data.


In lecture, we may have discussed the following important property of the mean:


The *sample mean* of data $X_1, X_2, \dots X_n$ is the unique minimizer $c$ of the function $$f(c)=\sum_{i=1}^n \left(X_i-c \right)^2. $$

The proof of that claim is as follows:

**Proof:**

Differentiating yields
$$f'(c)=\frac{df}{dc}\sum_{i=1}^n \left(X_i-c \right)^2 =\sum_{i=1}^n-2(X_i-c).$$ 

Setting $f'(c)=0$ gives

$$0=\sum_{i=1}^n-2(X_i-c)$$
$$=2nc-2\sum_{i=1}^n X_i$$
$$\implies\qquad  c=\frac{\sum_{i=1}^n X_i}{n}=\bar{X}$$

***

### Your exercise:

You are tasked with recreating a *similar* proof.  Prove the following:

The *median* of data $X_1, X_2, \dots X_n$ is the possibly non-unique minimizer $c$ of the function $$f(c)=\sum_{i=1}^n |X_i-c| $$

A few things to think about:

 - how do we differentiate the absolute value function?
 - what conditions might make the median non-unique in this case?  If it's nonunique, what possible values of $c$ still minimize the function $f$?

**Solution:**

The general format of the proof is the same as above, but now we have to differential the absolute value function.  Recall the definition of absolute value:


$$
|X_i-c| = \left\{
    \begin{array}\\
        X_i-c & \mbox{if } \ X_i-c \geq 0 \\
        -(X_i-c) & \mbox{if } X_i-c < 0 \\
    \end{array}
\right.
$$

If we're differentiating this, then, we have:
$$
\frac{d}{dc}|X_i-c| = \left\{
    \begin{array}\\
        -1 & \mbox{if } \ X_i-c \geq 0 \\
        1 & \mbox{if } X_i-c < 0 \\
    \end{array}
\right.
$$

Now we replicate the proof from above. Consider:

$$f(c)=\sum_{i=1}^n |X_i-c| $$

Differentiating yields
$$f'(c)=\frac{df}{dc}\sum_{i=1}^n |X_i-c| =\sum_{i=1}^n  K_i$$
where 
$$
K_i = \left\{
    \begin{array}\\
        -1 & \mbox{if } \ X_i-c \geq 0 \\
        1 & \mbox{if } X_i-c < 0 \\
    \end{array}
\right.
$$

Setting $f'(c)=0$ means we need to choose $c$ so that this sum is zero.  Since the sum is composed of entirely +1s and -1s, we need to choose $c$ so that there are equal numbers of +1s and -1s.  Thus:

The sum is minimized for all choices of $c$ with equally many data points $X_i>c$ as data points $X_i<c$.

If the $X_i$ values are sorted, it's the middle of the $X_i$ list: that's how we have defined the median, previously!

A couple of minor details:

 - The absolute value function is not actually differentiable at the values when $X_i=c$, since this would be a cusp.  This doesn't matter here, however, since we've trained to check those values for local min/max anyways.
 - If we have an odd number of data points, we must choose $c$ to be the exact "middle" point.  If this point is a *repeated* measurement, we should verify that choosing the middle observation still works: e.g. for data `X=[1,1,2]`, choosing a median of 1 is still the best choice.
 - If we have an even number of data points, e.g. for `X=[1,2]`, the distance-minimizer is *any* point between-and-including 1 or 2.  Choosing the average between those points is a statistical convention but not necessarily "optimal" my our data science measure!

[Back to top](#top)
<a id='p2'></a>

## (40 pts) Problem 2: Computation (Streaming Means)
***

Data science is often divided into two categories: questions of *what* the best value might be to repreesnt a data problem, and questions of *how* to compute that data value.  Question 1 - and prior lectures - should tell you that computing the mean is valuable!  But *how* do we compute the mean?

Let $x_1, x_2, \ldots, x_n$ be $n$ observations of a variable of interest.  Recall that the sample mean $\bar{x}_n$ and sample variance $s^2_n$ are given by 
<a id='eq1'></a>
$$
\bar{x}_n = \frac{1}{n}\sum_{k=1}^n x_k \quad \textrm{and} \quad s^2_n = \frac{1}{n-1}\sum_{k=1}^n \left( x_k - \bar{x}_n\right)^2 \qquad \tag{Equation 1}
$$

**Part A**:

How many computations - floating point operations: addition, subtraction, multiplication, division each count as 1 operation - are required to compute the mean of the data set with $n$ observations?


**Solution:**

Computing a mean with $n$ observations takes $n-1$ additions followed by a division, for $n$ operations total.

**Part B**:

Now suppose our data is *streaming*- we slowly add observations one at a time, instead of seeing the entire data set at once.  We are still interested in the mean, so if we stream the data set `[4,6,0,10, ...]`, we first compute the mean of the the first data point `[4]`, then we recompute the mean of the first two points `[4,6]`, then we recompute the mean of three `[4,6,0]`, and so forth.

Suppose we recompute the mean from scratch after each and every one of our $n$ observations are one-by-one added to our data set.  How many floating point operations are spent computing (and re-computing) the mean of the data set?

**Solution:**

 This means the first mean takes 1, the second mean takes 2, the third mean takes 3, and so forth.  Once we're computed $n$ sequential means, the total amount of computations is:

$1+2+3+\dots + n= \sum_{i=1}^n i = \frac{n(n+1)}{2}$ computations.  This is *order* $n^2$.

We should be convinced that streaming a mean costs a lot more computer time than just computing once!

In this problem we explore a smarter method for such an _online_ computation of the mean.  

**Result**: The following relation holds between the mean of the first $n-1$ observations and the mean of all $n$ observations: 

$$
\bar{x}_n = \bar{x}_{n-1} + \frac{x_n - \bar{x}_{n-1}}{n}
$$


A proof of this result is in the [Appendix](#Appendix) after problem 3, and requires some careful manipulations of the sum $\bar{x}_n$.  Your task will be to computationally verify and utilize this result.

**Part C**: Write a function `my_sample_mean` that takes as its input a numpy array and returns the mean of that numpy array using the formulas from class ([Equation 1](#eq1)). Write another function `my_sample_var` that takes as its input a numpy array and returns the variance of that numpy array, again using the formulas from class ([Equation 1](#eq1)). You may **not** use any built-in sample mean or variance functions.

In [18]:
#Solution:

def my_sample_mean(xx):
    n = len(xx)
    total = np.sum(xx)
    return total/n

def my_sample_var(xx):
    mean = my_sample_mean(xx)
    centered = xx - mean
    squared = centered**2
    return np.sum(squared)/(len(xx)-1)

**Part D**: Use your functions from Part B to compute the sample mean and sample variance of the following array, which contains the minutes late that the BuffBus is running on Friday afternoon.

`bus = [312, 4, 10, 0, 22, 39, 81, 19, 8, 60, 80, 42]`

In [19]:
bus = [312, 4, 10, 0, 22, 39, 81, 19, 8, 60, 80, 42]
mean = my_sample_mean(bus)
var = my_sample_var(bus)
print("Sample mean is: ", mean,"; sample variance is: ", var)

Sample mean is:  56.416666666666664 ; sample variance is:  7274.628787878787


**Part E**: Implement a third function called `update_mean` that implements the formula discussed after part B. Note that this function will need to take as its input three things: $x_n$, $\bar{x}_{n-1}$ and $n$, and returns $\bar{x}_{n}$. A function header and return statement are provided for you. This function may be auto-graded, so please do not change the given header API - the order of inputs matters! If you change it, you might lose points.

Use this function to compute the values that you get from taking the mean of the first buff buses' lateness, the first two buff buses' lateness, the first three buff buses' lateness, and so on up to all of the `bus` data points from **Part D**. Store your streaming bus means in a numpy array called `buffbus_bad_means`.  Report all 12 estimates in `buffbus_bad_means`.

In [20]:
# Given API:
def update_mean(prev_mean, xn, n):
    new_mean = prev_mean + (xn - prev_mean)/n
    return(new_mean)

buffbus_bad_means = []

prior_mean = bus[0]

length_of_loop = np.size(bus)
for i in range(length_of_loop):
    prior_mean = update_mean(prior_mean,bus[i],i+1)
    buffbus_bad_means.append(prior_mean)
buffbus_bad_means = np.array(buffbus_bad_means)

for idx,times in enumerate(buffbus_bad_means):
    print("The first {:.0f} busses have a mean lateness of {:.3f} minutes".format(idx+1,times))

The first 1 busses have a mean lateness of 312.000 minutes
The first 2 busses have a mean lateness of 158.000 minutes
The first 3 busses have a mean lateness of 108.667 minutes
The first 4 busses have a mean lateness of 81.500 minutes
The first 5 busses have a mean lateness of 69.600 minutes
The first 6 busses have a mean lateness of 64.500 minutes
The first 7 busses have a mean lateness of 66.857 minutes
The first 8 busses have a mean lateness of 60.875 minutes
The first 9 busses have a mean lateness of 55.000 minutes
The first 10 busses have a mean lateness of 55.500 minutes
The first 11 busses have a mean lateness of 57.727 minutes
The first 12 busses have a mean lateness of 56.417 minutes


**You may report any results for part E here, if not done using print() statements**

To ensure your function complies with the given API, run this small test, where we suppose we have a mean of $\bar{x}_n = 1$ with the first $2$ data points (`prev_mean`), and we update this with the 3rd ($n=3$) data point which is $x_3=2$:

In [21]:
assert update_mean(1,2,3)==4/3, "Warning: function seems broken."

**Part F**:

How many floating point operations were spent computing the final result in your code in **part E**?  Is this truly better than the uninformed approach from **part B**?

**Solution:**

The update_mean function took only 3 operations, and we perform it for all data points after the first ($n-1$ times total).  This means that for a data set that streams in $n$ data point, we preform $3(n-1)$ operations compared to the naive approach taking $\frac{n(n+1)}{2}$ operations.  This is an *order* $n$ operation instead of $n^2$, so the larger the data gets, the more time we save!

On our example with only 12 data points, the naive approach would perform 78 operations and the update_mean function just 33, so we're already saving plenty of time.


[Back to top](#top)

<a id='p3'></a>

## (40 pts) Problem 3: Data (Probability and Histograms)
*** 
The sinking of the RMS Titanic was a terrible tragedy that saw the loss of many lives. Even within this tragedy, thanks to the combinations of the records of the White Star Line and the thorough nature of follow-up research after the accident we have some records that can help us try to piece together the course of events on board the ship. Many of the historians and other researchers who have investigated this event have speculated as to what exactly happened.

We have the data on survival rates by class, gender, and age, so let's figure out whether there is evidence for some of these scenarios. Access the Titanic data in `titanic_data.csv` and store it in a Pandas DataFrame. The data contains information pertaining to class status (**Pclass**), survival (**Survived**), and gender (**Sex**) of passengers, among other things. Be sure to use the `titanic_data.csv` data set, *not* the `clean_titanic_data` file or `dirty_titanic_data` file from the in-class notebook exercises.

In [22]:
filepath = 'titanic_data.csv'
df = pd.read_csv(filepath)
df.head()
# print(df.keys())

FileNotFoundError: [Errno 2] File titanic_data.csv does not exist: 'titanic_data.csv'

**Part A**:
Based on the overall population of passengers, report the probability of survival.

$$P(Survived=1)$$

In [ ]:
print("Solution: " + str(df["Survived"].sum()/df["Survived"].count()))

**Part B**: 
Some claim that the final hours aboard the RMS Titanic were marked by "class warfare" in which the people with first-class tickets took all the good spots on the lifeboats; others claim that the final hours were characterized by male chivalry, in which the men valiantly gave up their positions in the boats and succumbed bravely to the depths of the Atlantic. 

Consider the two claims: class warfare, and male chivalry. Suppose that class warfare occurred in the final hours aboard the Titanic.  What patterns might you expect to see in the data?  Suppose that male chivalry was widespread during the final hours instead. What patterns might you then expect to see in the data?  Explain both of these hypothesized patterns in words. Are these two hypotheses mutually exclusive or not?

**SOLUTION**

If there were class warfare, we would expect that the first class is asserting its privilege over the lower classes, so first class passengers should be more likely to survive.  If there were chivalry, it would be the case that men have lower survival rates than women.  These are not mutually exclusive.

**Part C**: Use Pandas methods to create a clean data set by removing any rows from the DataFrame that are missing values corresponding to **Survived**, **Pclass**, **Age**, or **Sex**. Store the clean data in a DataFrame called dfTitanic. Be sure to show any exploratory work determining if/where there are rows with missing values. _HINT: There should be 714 rows in your cleaned data set._

In [ ]:
dfTitanic = df.dropna(subset=["Survived", "Pclass", "Age", "Sex"])
#dfTitanic.head()
dfTitanic.shape

**Part D**: Compute the probability of survival according to class, gender, and all combinations of the two variables.  Then, answer the following questions:
* **(i)** When reviewing class survival probability, how do the results compare to the base survival probability results from **Part A**?
* **(ii)** When reviewing gender survival probability, how do the results compare to the base survival probability results from **Part A**?
* **(iii)** Within each passenger class, were men or women more/less/equally likely to survive?
* **(iv)**  Did men in first class or women in third class have a higher survival probability?


In [ ]:
print("SOLUTION")

for sex in ["male", "female"]:
    survived = dfTitanic.loc[(dfTitanic["Sex"]==sex), "Survived"].sum()
    total = len(dfTitanic.loc[(dfTitanic["Sex"]==sex), "Survived"])
    print("{:>6} Survived: {:.0f}, Total: {:.0f}, Survival Fraction: {:.3f}".format(sex, survived, total, survived/total))

for pc in [1,2,3]:
    survived = dfTitanic.loc[(dfTitanic["Pclass"]==pc), "Survived"].sum()
    total = len(dfTitanic.loc[(dfTitanic["Pclass"]==pc), "Survived"])
    #print(pc, sex, survived, total)
    print("Class {} Survived: {:.0f}, Total: {:.0f}, Survival Fraction: {:.3f}".format(pc, survived, total, survived/total))
        
for pc in [1,2,3]:
    for sex in ["male", "female"]:
        survived = dfTitanic.loc[(dfTitanic["Pclass"]==pc) & (dfTitanic["Sex"]==sex), "Survived"].sum()
        total = len(dfTitanic.loc[(dfTitanic["Pclass"]==pc) & (dfTitanic["Sex"]==sex), "Survived"])
        #print(pc, sex, survived, total)
        print("Class {} {:>6} Survived: {:.0f}, Total: {:.0f}, Survival Fraction: {:.3f}".format(pc, sex, survived, total, survived/total))

In each class, women had more probability of survival compared to men. Women in third class had a higher probability of survival.

**Part E**: One might wonder how a passenger's age is related to the likelihood that they would survive the Titanic disaster. In addition to the "male chivalry" argument outlined above, you can perhaps imagine an addendum - "women and children first!" - as the cry to ring out across the decks. Or you might imagine the opposite - rather than "class warfare", it is simply healthy adults fighting to take lifeboat spots for themselves.

To answer this question graphically, plot two density histograms on the same set of axes, showing the distribution of the ages of passengers who survived, and the distribution of the ages of passengers who did not. 
* Use the bin edges $[0,5,10,\ldots,70,75,80]$ for both histograms.
* To better distinguish between our populations, we will represent survivors with `navy` (as they were eventually rescued by ships) and those who passed away with `sandybrown`.
* Plot both histograms on a single set of axes (there should be only one panel in the figure you create), but use Matplotlib/Pandas plotting functionality to make the faces of the histogram boxes somewhat transparent, so both histograms are visible.
* Include a legend and label your axes.
* Comment on the results. Does your figure suggest that some age ranges are more or less likely to have survived the disaster than other ages? Fully explain your reasoning and use your figure to justify your conclusions.
* If you noticed some relationship between age and likelihood of survival, what is one possible explanation?

In [ ]:
my_bins = range(0,80,5)
fig, ax = plt.subplots(figsize=(12,6))
dfTitanic.loc[dfTitanic["Survived"]==1].hist(column="Age", ax=ax, facecolor="navy", edgecolor="white",bins=my_bins, alpha=0.5, label="Survived", density=True)
dfTitanic.loc[dfTitanic["Survived"]==0].hist(column="Age", ax=ax, facecolor="sandybrown", edgecolor="white",bins=my_bins, alpha=0.3, label="Died", density=True)
ax.set_xlabel("Age", fontsize=16)
ax.set_ylabel("Frequency", fontsize=16)
ax.set_title("Passenger Age", fontsize=24)
ax.grid(alpha=0.25)
ax.set_axisbelow(True)
ax.legend();

The density histogram for passengers who died is taller (i.e., more likely) for passengers under the age of 15 and passengers in their 30s. So it appears children and adults in their 30s were more likely to perish, whereas young adults (15-30) seem to be more likely than not to survive. For adults over the age of 40, there does not seem to be any discernable trend.

**Part F:** In Part E, we plotted two *density* histograms, showing the distributions of ages of passengers that survived or did not survive the Titanic disaster. Why would it be misleading for us to have plotted these as *frequency* histograms instead?

**Solution:**

It would be misleading to plot these as frequency histograms because there are many more passengers who did not survive the disaster (about 2/3 of them), so the frequency histogram for that data subset would have more area than the frequency histogram for the surviving passengers.

**Part G**: Do the data suggest class warfare, male chivalry, age bias, or some combination of these characteristics in the final hours aboard the Titanic?  Justify your conclusions based on the computations done above, or do any other analysis that you like, but be sure to clearly justify your conclusions.

**EXAMPLE**
Women tended to survive more than men, so it appears that the men were being chivalrous and making sure that the women got off the boat first. On the other hand, the first class folks were far more likely to survive. We also saw some sort of age bias in survival for the "young adults", which could be due to a conscious attempt or it could be due to a simple healthiness effect (children and older folks may not have been able to handle the arctic waters). It could be a mix of all of these factors. However, it is also entirely possible that we need more information. For example, perhaps the person's location on the ship would be a good item to study in relation to survival.

**P.S.** It is not a component of your graded assignment, but the 1997 James Cameron film _Titanic_ captured some of these very notions in some riveting cinema. Whether or not you found evidence for these cases in the data, you might find it interesting to watch the movie (or just the segments of the sinking) to see one interpretation of these ideas. You can perhaps see how we might be persuaded to reinterpret the evidence of data by a heart-wrenching performance from a handsome young Leonardo DiCaprio!

<a id='Appendix'></a>

## Appendix 

*Goal*: Prove that 
$$
\bar{x}_n = \bar{x}_{n-1} + \frac{x_n - \bar{x}_{n-1}}{n}
$$

Note that you can get an expression for $\bar{x}_{n-1}$ by simply replacing $n$ in Equation 1 above with $n-1$.

We'll start with $\bar{x}_n$ and massage it until we get the righthand side of the formula

\begin{eqnarray}
\nonumber \bar{x}_n &=& \frac{1}{n} \sum_{k=1}^n x_k \\
&=& \frac{1}{n} \sum_{k=1}^{n-1} x_k + \frac{1}{n}x_n \\
&=& \frac{n-1}{n-1}\frac{1}{n} \sum_{k=1}^{n-1} x_k + \frac{1}{n}x_n \\
&=& \frac{n-1}{n} \left(\frac{1}{n-1} \sum_{k=1}^{n-1} x_k\right) + \frac{1}{n}x_n \\
&=& \frac{n-1}{n} \bar{x}_{n-1} + \frac{1}{n}x_n \\
&=& \frac{n}{n}\bar{x}_{n-1} - \frac{1}{n}\bar{x}_{n-1} + \frac{1}{n}x_n \\
&=&  \bar{x}_{n-1} + \frac{x_n - \bar{x}_{n-1}}{n} \quad \checkmark
\end{eqnarray}

